<a href="https://colab.research.google.com/github/gohar-malik/anomaly-det/blob/main/cifar103.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion * planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion * planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18(num_classes):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


def test():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())
import os
import sys
import argparse
import time
from datetime import datetime

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.utils.data import DataLoader

def train(epoch):

    start = time.time()
    net.train()
    total_loss = 0.0
    for batch_index, (images, labels) in enumerate(cifar10_training_loader):
        labels = labels.to(device)
        images = images.to(device)

        optimizer.zero_grad()
        outputs = net(images)
        loss = loss_function(outputs, labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    finish = time.time()
    print(f'Epoch: {epoch}')
    print(f'Training:\tLoss: {total_loss/len(cifar10_training_loader):0.4f}\tLR: {optimizer.param_groups[0]["lr"]:0.6f}\tTime: {finish-start:.2f}s')

@torch.no_grad()
def eval_training(epoch=0):

    start = time.time()
    net.eval()

    test_loss = 0.0 # cost function error
    correct = 0.0

    for (images, labels) in cifar10_test_loader:
        
        labels = labels.to(device)
        images = images.to(device)

        outputs = net(images)
        loss = loss_function(outputs, labels)

        test_loss += loss.item()
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum()

    finish = time.time()

    print(f'Testing:\tLoss: {test_loss / len(cifar10_test_loader.dataset):.4f}\tAcc: {correct.float() / len(cifar10_test_loader.dataset):.4f}\tTime:{ finish - start:.2f}s')

    return correct.float() / len(cifar10_test_loader.dataset)
parser = argparse.ArgumentParser()
parser.add_argument('-gpu', type=int, default=0, help='gpu id to use')
parser.add_argument('-b', type=int, default=128, help='batch size for dataloader')
parser.add_argument('-epochs', type=int, default=30, help='number of epochs to train')
parser.add_argument('-lr', type=float, default=0.05, help='initial learning rate')
parser.add_argument('-ckpt', default='./model_ResNet18_cifar10_b128_ep300_g0.1',help='directory of model for saving checkpoint')
parser.add_argument('-ckptepoch', type=int, default=1 ,help='directory of model for saving checkpoint')
parser.add_argument('--seed', type=int, default=42, help='random seed')
parser.add_argument("-f", required=False)
args, unknown = parser.parse_known_args()

### device config
use_cuda = (args.gpu is not None) and (torch.cuda.is_available())
torch.manual_seed(args.seed)
device = torch.device(f"cuda:{args.gpu}" if use_cuda else "cpu")
print(f"Using Device: {device}")

### network initialize
net = ResNet18(num_classes=10).to(device)

#### data loaders
mean = (0.49139968, 0.48215827 ,0.44653124)
std = (0.24703233,0.24348505,0.26158768)
transform_train = transforms.Compose([
    #transforms.ToPILImage(),
    transforms.RandomCrop(32, padding=2),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
#cifar10_training = CIFAR10Train(path, transform=transform_train)
cifar10_training = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
cifar10_training_loader = DataLoader(cifar10_training, shuffle=True, num_workers=2, batch_size=args.b)

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
#cifar10_test = CIFAR10Test(path, transform=transform_test)
cifar10_test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
cifar10_test_loader = DataLoader(cifar10_test, shuffle=True, num_workers=2, batch_size=args.b)

### training config
milestones = [150,225] #[60, 120, 160]
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=args.lr, momentum=0.9, weight_decay=0.1e-4) #2e-4
train_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=0.1) #0.2
iter_per_epoch = len(cifar10_training_loader)

### create checkpoint folder to save model
checkpoint_path = args.ckpt
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
checkpoint_path = os.path.join(checkpoint_path, '{epoch}_{type}.pth')

best_acc = 0.0
for epoch in range(1, args.epochs+1):

    train(epoch)
    acc = eval_training(epoch)
    train_scheduler.step(epoch)

    #start to save best performance model after learning rate decay to 0.01
    if epoch > milestones[0] and best_acc < acc:
        weights_path = checkpoint_path.format(epoch=epoch, type='best')
        print('saving weights file to {}'.format(weights_path))
        torch.save(net.state_dict(), weights_path)
        best_acc = acc
        continue

    if not epoch % args.ckptepoch:
        weights_path = checkpoint_path.format(epoch=epoch, type='regular')
        print('saving weights file to {}'.format(weights_path))
        torch.save(net.state_dict(), weights_path)
parser = argparse.ArgumentParser()
parser.add_argument('-weights', type=str,default="/content/model_ResNet18_cifar10_b128_ep300_g0.1/1_regular.pth", help='the weights file you want to test')
parser.add_argument('-gpu', type=int, default=0, help='gpu id to use')
parser.add_argument('-b', type=int, default=16, help='batch size for dataloader')
parser.add_argument('-ckpt', type=str, default='checkpoints/', help='checkpoint directory path')

args, unknown = parser.parse_known_args()

####### please add the path to the weights file here ######
args.weights = "/content/model_ResNet18_cifar10_b128_ep300_g0.1/1_regular.pth"

### device config
use_cuda = (args.gpu is not None) and (torch.cuda.is_available())
device = torch.device(f"cuda:{args.gpu}" if use_cuda else "cpu")
print(f"Using Device: {device}")

### network initialize
net = ResNet18(num_classes=10).to(device)

#### data loaders
mean = (0.49139968, 0.48215827 ,0.44653124)
std = (0.24703233,0.24348505,0.26158768)

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
cifar10_test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
cifar10_test_loader = DataLoader(cifar10_test, shuffle=True, num_workers=4, batch_size=args.b)

net.load_state_dict(torch.load(args.weights))
# print(net)
net.eval()

correct_1 = 0.0
correct_5 = 0.0
total = 0

with torch.no_grad():
    for n_iter, (image, label) in enumerate(tqdm(cifar10_test_loader)):
        # print("iteration: {}\ttotal {} iterations".format(n_iter + 1, len(cifar100_test_loader)))

        label = label.to(device)
        image = image.to(device)


        output = net(image)
        _, pred = output.topk(5, 1, largest=True, sorted=True)

        label = label.view(label.size(0), -1).expand_as(pred)
        correct = pred.eq(label).float()

        #compute top 5
        correct_5 += correct[:, :5].sum()

        #compute top1
        correct_1 += correct[:, :1].sum()

print()
print("Top 1 err: ", 1 - correct_1 / len(cifar10_test_loader.dataset))
print("Top 5 err: ", 1 - correct_5 / len(cifar10_test_loader.dataset))
print("Parameter numbers: {}".format(sum(p.numel() for p in net.parameters())))

Using Device: cuda:0


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch: 1
Training:	Loss: 1.6944	LR: 0.050000	Time: 50.22s
Testing:	Loss: 0.0112	Acc: 0.4743	Time:3.63s
saving weights file to ./model_ResNet18_cifar10_b128_ep300_g0.1/1_regular.pth


/usr/local/lib/python3.9/dist-packages/torch/optim/lr_scheduler.py:163: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch: 2
Training:	Loss: 1.2149	LR: 0.050000	Time: 43.08s
Testing:	Loss: 0.0082	Acc: 0.6353	Time:3.57s
saving weights file to ./model_ResNet18_cifar10_b128_ep300_g0.1/2_regular.pth
Epoch: 3
Training:	Loss: 0.9637	LR: 0.050000	Time: 43.00s
Testing:	Loss: 0.0066	Acc: 0.7019	Time:3.11s
saving weights file to ./model_ResNet18_cifar10_b128_ep300_g0.1/3_regular.pth
Epoch: 4
Training:	Loss: 0.8089	LR: 0.050000	Time: 43.02s
Testing:	Loss: 0.0065	Acc: 0.7248	Time:3.22s
saving weights file to ./model_ResNet18_cifar10_b128_ep300_g0.1/4_regular.pth
Epoch: 5
Training:	Loss: 0.6817	LR: 0.050000	Time: 42.96s
Testing:	Loss: 0.0054	Acc: 0.7757	Time:3.75s
saving weights file to ./model_ResNet18_cifar10_b128_ep300_g0.1/5_regular.pth
Epoch: 6
Training:	Loss: 0.5983	LR: 0.050000	Time: 43.11s
Testing:	Loss: 0.0046	Acc: 0.8005	Time:3.15s
saving weights file to ./model_ResNet18_cifar10_b128_ep300_g0.1/6_regular.pth
Epoch: 7
Training:	Loss: 0.5348	LR: 0.050000	Time: 43.07s
Testing:	Loss: 0.0042	Acc: 0.8189	Tim

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|██████████| 625/625 [00:06<00:00, 97.38it/s] 


Top 1 err:  tensor(0.5257, device='cuda:0')
Top 5 err:  tensor(0.0716, device='cuda:0')
Parameter numbers: 11173962
